# International Mouse Phenotyping Consortium (IMPC) Data API Workshop
Welcome to our workshop! In this session, we'll guide you through using Apache Solr API to access IMPC data. After that, we will focus on the phenodigm code. By the end, you'll confidently construct Solr queries to extract IMPC datasets. Get ready for hands-on exercises and real-world examples to reinforce your skills!

For more information about IMPC visit our [website](https://www.mousephenotype.org/).
Other useful links:
- International Mouse Phenotyping Resource of Standardised Screens | [IMPReSS](https://www.mousephenotype.org/impress/index)
- The Genome Targeting Repository | [GenTaR](https://www.gentar.org/tracker/#/)
- Workshop [repository](https://github.com/mpi2/impc-data-api-workshop/tree/main) with all materials
- IMPC Solr cores [documentation](https://www.ebi.ac.uk/mi/impc/solrdoc/)

# Set up
Let's start! First of all we need to import python libraries and set up helper function.
### Helper functions
Execute cell below. Follow steps:
1. Select cell by clicking into it.
2. Execute code by pressing ▷ play button above.
3. You can also use hotkey Ctrl + Enter to execute code.

In [ ]:
from IPython.display import display
from tqdm import tqdm
from urllib.parse import unquote

import csv
import pandas as pd
import requests

# Display the whole dataframe <100
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Create helper function
def solr_request(core, params, silent=False):
    """Performs a single Solr request.
    
    Returns:
        num_found: How many rows in total did the request match.
        df: A Pandas dataframe with a portion of the request matching `start` and `rows` parameters.
        silent: Suppress displaying the df and number of results (useful for batch requests).
    """
    base_url = "https://www.ebi.ac.uk/mi/impc/solr/"
    solr_url = base_url + core + "/select"

    response = requests.get(solr_url, params=params)
    if not silent:
        print(f"\nYour request:\n{unquote(response.request.url)}\n")
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        num_found = data["response"]["numFound"]
        if not silent:
            print(f'Number of found data points: {num_found}\n')
        # Extract and add search results to the list
        search_results = []
        for doc in data["response"]["docs"]:
            search_results.append(doc)
    
        # Convert the list of dictionaries into a DataFrame and print the DataFrame
        df = pd.DataFrame(search_results)
        if not silent:
            display(df)
        return num_found, df
    
    else:
        print("Error:", response.status_code, response.text)

### Example query
We will use `solr_request` function to access IMPC data using Solr API. Let's run cell below and investigate the result.  

In [ ]:
num_found, df = solr_request(
    core="genotype-phenotype",
    params={
        "q": "*:*",  # Your query, '*' retrieves all documents
        "rows": 10,  # Number of rows to retrieve
        "fl": "marker_symbol,allele_symbol,parameter_stable_ide",  # Fields to retrieve
        "wt": "json"  # Response format
    }
)

Let's take a look at the output of helper function. You can see following:
1. Submitted request, that you can open in browser by clicking into the link.
2. Number of data points in the requested dataframe.
3. Table with the results of your query. It will display less than 100 rows.

Let's get started with the exercises!

# Exercise block A

### Exercise 1: Getting Familiar with the Core
We will be working with genotype-phenotype core. To get yourself familiar with data, request 3 rows and all columns from this core.

In [ ]:
num_found, df = solr_request(
    core=...,
    params={
        "q": ...,
        "rows": ...,
    }
)

### Exercise 2: Selecting Specific Columns
As you can see, there is a lot of columns. To focus on the columns we need, request only the following once:
- marker_symbol
- marker_accession_id
- zygosity
- parameter_name
- parameter_stable_id
- p_value
- mp_term_id
- mp_term_name
<br>

Modify query from exercise 1 to request limited list of the columns above. 

In [ ]:
num_found, df = solr_request(
    core="genotype-phenotype",
    params={
        "q": ...,
        "rows": ...,
        "fl": ...
    }
)

# Exercise block B
### Exercise 3: Filtering by Single Column
Let's now focus on a particular gene. In this example we will be using Dclk1. Filter the results so there only records of this gene are displayed by modifying query from exercise 2.

In [ ]:
num_found, df = solr_request(
    core="genotype-phenotype",
    params={
        "q": ...,
        "rows": ...,
        "fl": "marker_symbol,marker_accession_id,zygosity,parameter_name,parameter_stable_id,p_value,mp_term_id,mp_term_name"
    }
)

### Exercise 4: Filtering Numerical Values and Applying Multiple Filters
As you can see, not all values are statistically significant. In addition to the gene name filter, let's also filter by the p-value columnm so that it is less than 1e-4.
Modifying query from exercise 3 and display 10 rows instead of 3.

In [ ]:
num_found, df = solr_request(
    core="genotype-phenotype",
    params={
        "q": ...,
        "rows": ...,
        "fl": "marker_symbol,marker_accession_id,zygosity,parameter_name,parameter_stable_id,p_value,mp_term_id,mp_term_name"
    }
)

We got the result! 
1. Navigate to the [IMPReSS website](https://www.mousephenotype.org/impress/index).
2. Search for the parameter_stable_id `IMPC_GRS_010_001`, which is referenced in Exercise 4.
3. Examine the procedure associated with the procedure key `IMPC_GRS_001` by clicking on the "View procedure" button.
4. Within the "Parameters & Metadata" section, locate the parameter `IMPC_GRS_010_001`.
5. Answer the following question: What is the name of the parameter with the parameter_stable_id `IMPC_GRS_010_001`?

# Exercise block C

### Exercise 5: Downloading data in chunks

Download the data for `anatomy` core.

In [ ]:
def batch_request(core, params, batch_size):
    """Calls `solr_request` multiple times with `params` to retrieve results in chunk `batch_size` rows at a time."""
    if "rows" in "params":
        print("WARN: You have specified the `params` -> `rows` value. It will be ignored, because the data is retrieved `batch_size` rows at a time.")
    # Determine the total number of rows. Note that we do not request any data (rows = 0).
    num_results, _ = solr_request(core=core, params={**params, "start": 0, "rows": 0}, silent=True)
    # Initialise everything for data retrieval.
    start = 0
    chunks = []
    # Request chunks until we have complete data.
    with tqdm(total=num_results) as pbar:  # Initialize tqdm progress bar.
        while start < num_results:
            # Update progress bar with the number of rows requested.
            pbar.update(batch_size) 
            # Request chunk. We don't need num_results anymore because it does not change.
            _, df_chunk = solr_request(core=core, params={**params, "start": start, "rows": batch_size}, silent=True)
            # Record chunk.
            chunks.append(df_chunk)
            # Increment start.
            start += batch_size
    # Prepare final dataframe.
    return pd.concat(chunks, ignore_index=True)

In [ ]:
# Request dataframe in chunks.
df = batch_request(
    core=...,
    params={
        "q": "*:*",
    },
    batch_size=100
)

In [ ]:
# Save dataframe to JSON (lines) format for subsequent work. This will contain a single self contained JSON record per line.
df.to_json("anatomy_df.json", orient="records", lines=True)

In [ ]:
# We can also save as CSV, but note that fine structure such as lists and nested data will be lost.
df.to_csv("anatomy_df.csv", index=False)

# Exercise block D

### Exercise 6: Faceting Query
In this exercise we will be again querying the whole core. We want to count how many records there are for each value of the `zygosity` column.

In [ ]:
def facet_request(core, params, silent=False):
    """Performs a single Solr request.
    
    Returns:
        num_found: How many rows in total did the request match.
        df: A Pandas dataframe with a portion of the request matching `start` and `rows` parameters.
        silent: Suppress displaying the df and number of results (useful for batch requests).
    """
    base_url = "https://www.ebi.ac.uk/mi/impc/solr/"
    solr_url = base_url + core + "/select"

    response = requests.get(solr_url, params=params)
    if not silent:
        print(f"\nYour request:\n{unquote(response.request.url)}\n")
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        num_found = data["response"]["numFound"]
        if not silent:
            print(f'Number of found data points: {num_found}\n')
        # Extract and add faceting query results to the list
        facet_counts = data["facet_counts"]["facet_fields"][params["facet.field"]]
        # Initialize an empty dictionary
        faceting_dict = {}
        # Iterate over the list, taking pairs of elements
        for i in range(0, len(facet_counts), 2):
            # Assign label as key and count as value
            label = facet_counts[i]
            count = facet_counts[i + 1]
            faceting_dict[label] = [count]
        
        # Print the resulting dictionary
        # Convert the list of dictionaries into a DataFrame and print the DataFrame
        df = pd.DataFrame(faceting_dict)
        df = pd.DataFrame.from_dict(faceting_dict, orient='index', columns=['counts']).reset_index()

        # Rename the columns
        df.columns = [params["facet.field"], 'count_per_category']
        if not silent:
            display(df)
        return num_found, df
    
    else:
        print("Error:", response.status_code, response.text)

In [ ]:
num_found, df = facet_request(
    core="genotype-phenotype",
    params={
        "q": "*:*",
        "rows": 0,
        "facet": "on",
        "facet.field": ...
    }
)

# Exercises: Phenodigm Core
### Exercise 1: Getting familiar with the Phenodigm core
Get familiar with the phenodigm query structure. Write a query to see disease model summary.

In [ ]:
num_found, df = solr_request(
    core=...,
    params={
        "q": "type=disease_model_summary",
        "rows": 10
    }
)

### Exercise 2: Search by Disease
How many entries are there on Robinow Syndrome (OMIM:618529)?

In [ ]:
num_found, df = solr_request(
    core="phenodigm",
    params={
        "q": ...,
        "rows": 10,
    }
)

### Exercise 3: Search all models related to the disease
Get all models (and their info) related to a disease (OMIM:618529)
- Some useful fields are:
    - model_id
    - disease_id
    - marker_id
    - marker_symbol
    - model_source
    - model_genetic_background

In [ ]:
num_found, df = solr_request(
    core="phenodigm",
    params={
        "q": 'type:disease_model_summary AND disease_id:"OMIM:618529"',
        "rows": 10,
        "fl": ...
    }
)

### Exercise 4: Search by disease and mouse gene
Look for entries that contain both the mouse gene *Nxn* (MGI:109331) and the disease Robinow syndrome (OMIM:618529).

In [ ]:
num_found, df = solr_request(
    core="phenodigm",
    params={
        "q":...,
        "rows":10,
    }
)

### Exercise 5: Calculate the Phenodigm score
Get all diseases related to *Nxn* (MGI:109331), calculate their **phenodigm score** and sort it in descending manner
- Select the following fields:
    - disease_id
    - disease_term
    - disease_model_avg_norm
    - disease_model_max_norm

To calculate the phenodigm score add this after your last field:
`phenodigm_score:div(sum(disease_model_avg_norm,disease_model_max_norm),2)`

In [ ]:
num_found, df = solr_request(
    core="phenodigm",
    params={
        "rows": 10,
        "q": 'type:disease_model_summary AND marker_id:"MGI:109331"',
        "fl": ...
    }
)

### Exercise 6: Iterate over a list of diseases or models/ genes
Now we define another helper function which will request information for a list of values of a given entity. This is useful when we want to request, for example, data for specific models or genes. This is how the function is defined:

In [ ]:
## Successful attempt at using a generator and write a csv to iterate over a list of genes!

def entity_iterator(base_url, params):
    """Generator function to fetch results in chunks using pagination"""
    while True:
        response = requests.get(base_url, params=params)
        data = response.json()
        docs = data['response']['docs']

        # Yield the documents in the current chunk
        for doc in docs:
            yield doc

        # Check if there are more results to fetch
        start = params['start'] + params['rows']
        num_found = data['response']['numFound']
        if start >= num_found:
            break
        

        # Update the start parameter for the next request
        params['start'] = start
        
    # Print number of found data points
    print(f'Number of found data points: {data["response"]["numFound"]}\n')

    
def iterator_solr_request(core, params, filename):
    """ Function to fetch results in batches from the Solr API and write them to a csv file"""
    # Base URL
    base_url = "https://www.ebi.ac.uk/mi/impc/solr/"
    solr_url = base_url + core + "/select"

    # Extract entities_list and entity_type from params
    entity_list = params.pop('entity_list')
    entity_type = params.pop('entity_type')

    # Construct the filter query with grouped model IDs
    # TODO: this is not very readable but it seems using f-strings and backslashes needed for entities is an issue
    fq = '{}:({})'.format(entity_type, ' OR '.join(['"{}"'.format(id) for id in entity_list]))

    print('Query: ',fq)
    params['fq'] = fq

    # Fetch results using a generator function
    results_generator = entity_iterator(solr_url, params)

    # Open the file in write mode
    with open(filename, 'w', newline='') as f:
        writer = None
        for item in results_generator:
            # Initialize the CSV writer with the keys of the first item as the field names
            if writer is None:
                writer = csv.DictWriter(f, fieldnames=item.keys())
                writer.writeheader()
            # Write the item to the CSV file
            writer.writerow(item)
        print(f'File {filename} was created.')

And this is how we can use it:

In [ ]:
# List of model IDs.
models = ["MGI:3587188","MGI:3587185","MGI:3605874","MGI:2668213"]

# Initial query parameters.
params = {
    'q': 'type:disease_model_summary',
    'rows': 100,  # Fetch results in chunks of 100
    'wt': 'json',  # Receive response in JSON format
    'fl': 'model_id,marker_id,disease_id',
    'start': 0,  # Start from the beginning
    'entity_list': models,
    'entity_type': "model_id"
}

iterator_solr_request(core="phenodigm", params=params, filename='model_ids.csv')

In [ ]:
df = pd.read_csv('model_ids.csv')

Now, make your own request based on the example above, with the following changes:
* Use the `genotype-phenotype` core
* Iterate over genes (the name of the field is `marker_symbol`)
* Use the following list of genes: _Zfp580, Firrm, Gpld1, Mbip_
* Modify the list of fields you request (`fl`) to get marker symbol, allele symbol, and parameter stable ID

In [ ]:
# Genes example
genes = ...

# Initial query parameters
params = {
    'q': "*:*",
    'rows': 100,  # Fetch results in chunks of 100
    'wt': 'json',  # Receive response in JSON format
    "fl": ...,
    'start': 0,  # Start from the beginning
    'entity_list': genes,
    'entity_type': ...
}

iterator_solr_request(core=..., params=params, filename='marker_symbol.csv')